In [19]:
import numpy as np


class LogisticRegression():
    @staticmethod
    def loss(theta, x, y, lambda_param=None):
        """Loss function for logistic regression with without regularization"""
        exponent = - y * (x.dot(theta))
        return np.sum(np.log(1+np.exp(exponent))) / x.shape[0]

    @staticmethod
    def gradient(theta, x, y, lambda_param=None):
        """
        Gradient function for logistic regression without regularization.
        Based on the above logistic_regression
        """
        exponent = y * (x.dot(theta))
        gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (
            x.shape[0])

        # Reshape to handle case where x is csr_matrix
        gradient_loss.reshape(theta.shape)

        return gradient_loss


class LogisticRegressionSinglePoint():
    @staticmethod
    def loss(theta, xi, yi, lambda_param=None):
        exponent = - yi * (xi.dot(theta))
        return np.log(1 + np.exp(exponent))

    @staticmethod
    def gradient(theta, xi, yi, lambda_param=None):

        # Based on page 22 of
        # http://www.cs.rpi.edu/~magdon/courses/LFD-Slides/SlidesLect09.pdf
        exponent = yi * (xi.dot(theta))
        return - (yi*xi) / (1+np.exp(exponent))


class LogisticRegressionRegular():
    @staticmethod
    def loss(theta, x, y, lambda_param):
        regularization = (lambda_param/2) * np.sum(theta*theta)
        return LogisticRegression.loss(theta, x, y) + regularization

    @staticmethod
    def gradient(theta, x, y, lambda_param):
        regularization = lambda_param * theta
        return LogisticRegression.gradient(theta, x, y) + regularization

In [5]:
'''
Functions from in-class exercises
'''
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

In [6]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [7]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


In [8]:
'''
Functions taken from in-class-exercise 10.28.24
'''
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias) #this is the dot product and take the sign. 
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2) #computing L2 norm 
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def gradient_sum(theta, X, y, b):
    gradients = [L2_clip(gradient(theta, x_i, y_i), b) for x_i, y_i in zip(X,y)]
        
    # sum query
    # L2 sensitivity is b (by clipping performed above)
    return np.sum(gradients, axis=0)
#theta = [-.1 for _ in range(104)]
#accuracy(theta)

### IMPLEMENTING BATCH GRADIENT DESCENT ###

In [164]:
import numpy as np


class LogisticRegression():
    @staticmethod
    def loss(theta, x, y, lambda_param=None):
        """Loss function for logistic regression with without regularization"""
        exponent = - y * (x.dot(theta))
        return np.sum(np.log(1+np.exp(exponent))) / x.shape[0]

    @staticmethod
    def gradient(theta, x, y, lambda_param=None):
        """
        Gradient function for logistic regression without regularization.
        Based on the above logistic_regression
        """
        exponent = y * (x.dot(theta))
        gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (
            x.shape[0])

        # Reshape to handle case where x is csr_matrix
        gradient_loss.reshape(theta.shape)

        return gradient_loss

def batch_gradient_descent(epochs, n):
    
    theta = np.zeros(X_train.shape[1])
    regression = LogisticRegression()
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None) 

        # Update the parameters (theta)
        theta -= n * grad

    #print(theta)
    return theta

theta0 = batch_gradient_descent(20, .1)
theta1 = batch_gradient_descent(40, .1)
theta2 = batch_gradient_descent(60, .1)
theta3 = batch_gradient_descent(80, .1)
theta4 = batch_gradient_descent(100, .1)
theta5 = batch_gradient_descent(20, 1)
theta6 = batch_gradient_descent(40, 1)
theta7 = batch_gradient_descent(60, 1)
theta8 = batch_gradient_descent(80, 1)
theta9 = batch_gradient_descent(100, 1)

print('Final accuracy with epochs = 20, learning rate = .1:  ', accuracy(theta0))
print('Final accuracy with epochs = 40, learning rate = .1:  ', accuracy(theta1))
print('Final accuracy with epochs = 60, learning rate = .1:  ', accuracy(theta2))
print('Final accuracy with epochs = 80, learning rate = .1:  ', accuracy(theta3))
print('Final accuracy with epochs = 100, learning rate = .1:  ', accuracy(theta4))
print('Final accuracy with epochs = 20, learning rate = 1:  ', accuracy(theta5))
print('Final accuracy with epochs = 40, learning rate = 1:  ', accuracy(theta6))
print('Final accuracy with epochs = 60, learning rate = 1:  ', accuracy(theta7))
print('Final accuracy with epochs = 80, learning rate = 1:  ', accuracy(theta8))
print('Final accuracy with epochs = 100, learning rate = 1:  ', accuracy(theta9))

Final accuracy with epochs = 20, learning rate = .1:   0.7585139318885449
Final accuracy with epochs = 40, learning rate = .1:   0.7587350729765591
Final accuracy with epochs = 60, learning rate = .1:   0.7628261831048209
Final accuracy with epochs = 80, learning rate = .1:   0.7778637770897833
Final accuracy with epochs = 100, learning rate = .1:   0.7807386112339673
Final accuracy with epochs = 20, learning rate = 1:   0.8081601061477223
Final accuracy with epochs = 40, learning rate = 1:   0.8159000442282176
Final accuracy with epochs = 60, learning rate = 1:   0.8182220256523662
Final accuracy with epochs = 80, learning rate = 1:   0.8209862892525431
Final accuracy with epochs = 100, learning rate = 1:   0.8228659885006634


### EPSILON DP ###

In [40]:
def batch_gradient_epsilon_descent(epochs, epsilon, n):
    
    theta = np.zeros(X_train.shape[1])
    regression = LogisticRegression()
    epsilon_i = epsilon/epochs
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        grad_noisy = laplace_mech_vec(grad, sensitivity=1, epsilon=epsilon_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    return theta
    
theta0 = batch_gradient_epsilon_descent(10, .01, 1)
theta1 = batch_gradient_epsilon_descent(10, .1, 1)
theta2 = batch_gradient_epsilon_descent(10, .5, 1)
theta3 = batch_gradient_epsilon_descent(10, 1, 1)
theta4 = batch_gradient_epsilon_descent(10, 5, 1)
theta5 = batch_gradient_epsilon_descent(10, 10, 1)

print('Final accuracy with epsilon = .01:  ', accuracy(theta0))
print('Final accuracy with epsilon = .1:  ', accuracy(theta1))
print('Final accuracy with epsilon = 1:  ', accuracy(theta2))
print('Final accuracy with epsilon = 10:  ', accuracy(theta3))
print('Final accuracy with epsilon = 100:  ', accuracy(theta4))
print('Final accuracy with epsilon = 10 and epochs = 20:  ', accuracy(theta5))

C:\Users\darkf\AppData\Local\Temp\ipykernel_41276\321558702.py:18: RuntimeWarning: overflow encountered in exp
  gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (


Final accuracy with epsilon = .01:   0.2966607695709863
Final accuracy with epsilon = .1:   0.5413533834586466
Final accuracy with epsilon = 1:   0.5682220256523662
Final accuracy with epsilon = 10:   0.7307607253427687
Final accuracy with epsilon = 100:   0.3711853162317559
Final accuracy with epsilon = 10 and epochs = 20:   0.6551304732419283


### EPSILON DELTA DP ###

In [48]:
def batch_gradient_epsilon_delta_descent(epochs, epsilon, delta, n):
    
    theta = np.zeros(X_train.shape[1])
    regression = LogisticRegression()
    epsilon_i = epsilon/epochs
    delta_i = delta/epochs
    b=3
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        clipped_grad = L2_clip(grad, b)
        grad_noisy = gaussian_mech_vec(clipped_grad, sensitivity=b, epsilon=epsilon_i, delta = delta_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    #print(theta)
    return theta

theta0 = batch_gradient_epsilon_delta_descent(20, 1, 1e-5, 1)
theta1 = batch_gradient_epsilon_delta_descent(20, 10, 1e-5, 1)
theta2 = batch_gradient_epsilon_delta_descent(10, .5, 1e-5, 1)
theta3 = batch_gradient_epsilon_delta_descent(10, 1, 1e-5, 1)
theta4 = batch_gradient_epsilon_delta_descent(10, 5, 1e-5, 1)
theta5 = batch_gradient_epsilon_delta_descent(10, 10, 1e-5, 1)

print('Final accuracy with epsilon = 1 and epochs = 100:  ', accuracy(theta0))
print('Final accuracy with epsilon = 10 and epochs = 100:   ', accuracy(theta1))
print('Final accuracy with epsilon = .5 and epochs = 20:  ', accuracy(theta2))
print('Final accuracy with epsilon = 1 and epochs = 20:  ', accuracy(theta3))
print('Final accuracy with epsilon = 5 and epochs = 20:  ', accuracy(theta4))
print('Final accuracy with epsilon = 10 and epochs = 20:  ', accuracy(theta5))

C:\Users\darkf\AppData\Local\Temp\ipykernel_41276\321558702.py:18: RuntimeWarning: overflow encountered in exp
  gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (


Final accuracy with epsilon = 1 and epochs = 100:   0.33757187085360457
Final accuracy with epsilon = 10 and epochs = 100:    0.3455329500221141
Final accuracy with epsilon = .5 and epochs = 20:   0.5605926581158779
Final accuracy with epsilon = 1 and epochs = 20:   0.7113003095975232
Final accuracy with epsilon = 5 and epochs = 20:   0.7551968155683326
Final accuracy with epsilon = 10 and epochs = 20:   0.6884122069880584


### RDP ###

In [160]:
def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def batch_gradient_rdp_descent(epochs, epsilon_bar, alpha, n):
    
    theta = np.zeros(X_train.shape[1])
    
    regression = LogisticRegression()
    
    epsilon_i = epsilon_bar/epochs
    alpha_i = alpha/epochs
    b=3
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        clipped_grad = L2_clip(grad, b)
        grad_noisy = gaussian_mech_RDP_vec(clipped_grad, sensitivity=b, alpha=alpha_i, epsilon=epsilon_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    #print(theta)
    return theta

theta0 = batch_gradient_rdp_descent(10, .01, 3, 1)
theta1 = batch_gradient_rdp_descent(10, .1, 3, 1)
theta2 = batch_gradient_rdp_descent(10, 2, 3, 1)
theta3 = batch_gradient_rdp_descent(10, .01, 5, 1)
theta4 = batch_gradient_rdp_descent(10, .1, 5, 1)
theta5 = batch_gradient_rdp_descent(10, 2, 5, 1)

print('Final accuracy with epsilon = .01 and epochs = 50:  ', accuracy(theta0))
print('Final accuracy with epsilon = .1:  ', accuracy(theta1))
print('Final accuracy with epsilon = 1:  ', accuracy(theta2))
print('Final accuracy with epsilon = 10:  ', accuracy(theta3))
print('Final accuracy with epsilon = 100:  ', accuracy(theta4))
print('Final accuracy with epsilon = 10 and epochs = 50:  ', accuracy(theta5))

C:\Users\darkf\AppData\Local\Temp\ipykernel_41276\321558702.py:18: RuntimeWarning: overflow encountered in exp
  gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (


Final accuracy with epsilon = .01 and epochs = 50:   0.5515258735072976
Final accuracy with epsilon = .1:   0.4399601946041575
Final accuracy with epsilon = 1:   0.7058823529411765
Final accuracy with epsilon = 10:   0.5139318885448917
Final accuracy with epsilon = 100:   0.5218929677134011
Final accuracy with epsilon = 10 and epochs = 50:   0.718266253869969


In [ ]:
#taken from class
def rdp_convert(alpha, epsilon_bar, delta): #return epsilon
    return epsilon_bar + np.log(1/delta) / (alpha - 1)

rdp_convert(3, 20, 1e-5)

### zCDP ###

In [106]:
def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def batch_gradient_zcdp_descent(epochs, rho, n):
    
    theta = np.zeros(X_train.shape[1])
    
    regression = LogisticRegression()
    
    rho_i = rho/epochs
    b=3
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        clipped_grad = L2_clip(grad, b)
        grad_noisy = gaussian_mech_zCDP_vec(clipped_grad, sensitivity=b, rho=rho_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    #print(theta)
    return theta

theta0 = batch_gradient_zcdp_descent(10, .001, 1)
theta1 = batch_gradient_zcdp_descent(10, .01, 1)
theta2 = batch_gradient_zcdp_descent(10, .1, 1)
theta3 = batch_gradient_zcdp_descent(10, 1, 1)
theta4 = batch_gradient_zcdp_descent(10, 2, 1)
theta5 = batch_gradient_zcdp_descent(10, 10, 1)

print('Final accuracy with epsilon = .01:  ', accuracy(theta0))
print('Final accuracy with epsilon = .1:  ', accuracy(theta1))
print('Final accuracy with epsilon = 1:  ', accuracy(theta2))
print('Final accuracy with epsilon = 10:  ', accuracy(theta3))
print('Final accuracy with epsilon = 100:  ', accuracy(theta4))
print('Final accuracy with epsilon = 10 and epochs = 20:  ', accuracy(theta5))

C:\Users\darkf\AppData\Local\Temp\ipykernel_41276\321558702.py:18: RuntimeWarning: overflow encountered in exp
  gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (


Final accuracy with epsilon = .01:   0.6939407341884122
Final accuracy with epsilon = .1:   0.5856921716054843
Final accuracy with epsilon = 1:   0.6377708978328174
Final accuracy with epsilon = 10:   0.2951127819548872
Final accuracy with epsilon = 100:   0.6843210968597966
Final accuracy with epsilon = 10 and epochs = 20:   0.7235736399823087


In [142]:
#taken from class
def zcdp_convert(rho, delta):
    return rho + 2 * np.sqrt(rho * np.log(1 / delta))

zcdp_convert(2, 1e-5)

11.597051824376162

In [ ]:
# Moving forward - need to convert to epsilon-delta for zCDP and RDP to determine 
# which produces the best accuracy

In [170]:
ep_avg = np.mean([accuracy(batch_gradient_epsilon_descent(10, 5, 1)) for i in range(20)])
delta_avg = np.mean([accuracy(batch_gradient_epsilon_delta_descent(10, 5, 1e-5, 1)) for i in range(20)])
renyi_avg = np.mean([accuracy(batch_gradient_rdp_descent(10, 2, 3, 1)) for i in range(20)])
zcdp_avg = np.mean([accuracy(batch_gradient_zcdp_descent(10, 10, 1)) for i in range(20)])

print(ep_avg)
print(delta_avg)
print(renyi_avg)
print(zcdp_avg)

C:\Users\darkf\AppData\Local\Temp\ipykernel_41276\2581392234.py:18: RuntimeWarning: overflow encountered in exp
  gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (


0.5345975232198144
0.46572313135780624
0.5641585581601062
0.6046826625386996
